# Import Library


In [497]:
import pandas as pd
import numpy as np

# Load Data

In [498]:
df_users = pd.read_csv("clean_users.csv")
df_tracking = pd.read_csv("clean_tracking.csv")
df_submission = pd.read_csv("clean_submissions.csv")
df_complete = pd.read_csv("clean_completions.csv")
df_regist = pd.read_csv("clean_exam_registrations.csv")
df_exam = pd.read_csv("clean_exam_results.csv")
df_journey = pd.read_csv("developer_journeys.csv")

In [499]:
for name, df in {
    "Users": df_users,
    "Tracking": df_tracking,
    "Submission": df_submission,
    "Completion": df_complete,
    "Registration": df_regist,
    "Exam Results": df_exam,
    "Developer Journey": df_journey
}.items():

    print(f"\n===== {name} =====")
    print(df.info())



===== Users =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       31 non-null     int64  
 1   display_name  31 non-null     object 
 2   name          31 non-null     object 
 3   email         31 non-null     object 
 4   phone         30 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.3+ KB
None

===== Tracking =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101736 entries, 0 to 101735
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               101736 non-null  int64 
 1   journey_id       101736 non-null  int64 
 2   tutorial_id      101736 non-null  int64 
 3   developer_id     101736 non-null  int64 
 4   status           101736 non-null  int64 
 5   last_viewed      11885 non-null   object
 6   first_

# Normalisasi Primary Key untuk Tahap Merging

In [500]:
df_users.rename(columns={"id": "user_id"}, inplace=True)
df_tracking.rename(columns={"developer_id": "user_id"}, inplace=True)
df_submission.rename(columns={"submitter_id": "user_id"}, inplace=True)
df_complete.rename(columns={"user_id": "user_id"}, inplace=True)
df_regist.rename(columns={"examinees_id": "user_id"}, inplace=True)
df_exam.rename(columns={"exam_registration_id": "regist_id"}, inplace=True)
df_journey.rename(columns={"id": "journey_id"}, inplace=True)

# **1. Mencari Total dan Average Setiap User**

## **Mencari Summary**

### Tracking Summary

In [501]:
df_tracking["completed_at"] = pd.to_datetime(df_tracking["completed_at"], errors="coerce")

tracking_summary = (
    df_tracking
    .groupby("user_id")
    .agg(
        total_tracking_events=("tutorial_id", "count"),
        total_completed_modules=("completed_at", lambda x: x.notna().sum())
    )
    .reset_index()
)

### Submission SUmmary

In [502]:
df_submission["rating"] = pd.to_numeric(df_submission["rating"], errors="coerce")

submission_summary = (
    df_submission
    .groupby("user_id")
    .agg(
        total_submissions=("id", "count"),
        avg_submission_rating=("rating", "mean")
    )
    .reset_index()
)

### Completion SUmmary

In [503]:
df_complete["study_duration"] = pd.to_numeric(df_complete["study_duration"], errors="coerce")
df_complete["avg_submission_rating"] = pd.to_numeric(df_complete["avg_submission_rating"], errors="coerce")

In [504]:
completion_summary = (
    df_complete
    .groupby("user_id")
    .agg(
        avg_study_duration=("study_duration", "mean"),
        avg_completion_rating=("avg_submission_rating", "mean")
    )
    .reset_index()
)

### Exam Score Summary

In [505]:
df_exam["regist_id"] = df_exam["regist_id"].astype(str)
df_regist["id"] = df_regist["id"].astype(str)

df_exam["score"] = pd.to_numeric(df_exam["score"], errors="coerce")
df_exam["is_passed"] = pd.to_numeric(df_exam["is_passed"], errors="coerce")

In [506]:
df_exam_matched = df_exam.merge(
    df_regist,
    left_on="created_at",
    right_on="exam_finished_at",
    how="left"
)

In [507]:
df_exam_matched["score"] = pd.to_numeric(df_exam_matched["score"], errors="coerce")
df_exam_matched["is_passed"] = pd.to_numeric(df_exam_matched["is_passed"], errors="coerce")

exam_summary = (
    df_exam_matched
    .groupby("user_id")
    .agg(
        avg_exam_score=("score", "mean"),
        exam_pass_rate=("is_passed", "mean")
    )
    .reset_index()
)

# Pastikan user_id konsisten tipe Int
exam_summary["user_id"] = exam_summary["user_id"].astype("Int64")

### LAst Activity Summary

In [508]:
df_tracking["last_viewed"] = pd.to_datetime(df_tracking["last_viewed"], errors="coerce")
df_tracking["first_opened_at"] = pd.to_datetime(df_tracking["first_opened_at"], errors="coerce")
df_tracking["completed_at"] = pd.to_datetime(df_tracking["completed_at"], errors="coerce")

df_tracking["last_activity_date"] = df_tracking[
    ["last_viewed", "first_opened_at", "completed_at"]
].max(axis=1)

In [509]:
# Ambil aktivitas terakhir per user
last_active_summary = (
    df_tracking
    .groupby("user_id")["last_activity_date"]
    .max()
    .reset_index()
)

In [510]:
# Hitung selisih hari dari "hari ini"
today = pd.Timestamp.today().normalize()
last_active_summary["days_since_last_active"] = (
    today - last_active_summary["last_activity_date"]
).dt.days

## **Merging Data Summary**

### Menggabunbgkan Data Aggregasi

In [511]:
df_users["user_id"] = df_users["user_id"].astype(int)
tracking_summary["user_id"] = tracking_summary["user_id"].astype(int)
submission_summary["user_id"] = submission_summary["user_id"].astype(int)
completion_summary["user_id"] = completion_summary["user_id"].astype(int)
exam_summary["user_id"] = exam_summary["user_id"].astype(int)
last_active_summary["user_id"] = last_active_summary["user_id"].astype(int)

In [512]:
df_final = (
    df_users
    .merge(tracking_summary, on="user_id", how="left")
    .merge(submission_summary, on="user_id", how="left")
    .merge(completion_summary, on="user_id", how="left")
    .merge(exam_summary, on="user_id", how="left")
    .merge(last_active_summary[["user_id", "days_since_last_active"]],
           on="user_id", how="left")
)

In [513]:
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  31 non-null     int64  
 1   display_name             31 non-null     object 
 2   name                     31 non-null     object 
 3   email                    31 non-null     object 
 4   phone                    30 non-null     float64
 5   total_tracking_events    31 non-null     int64  
 6   total_completed_modules  31 non-null     int64  
 7   total_submissions        26 non-null     float64
 8   avg_submission_rating    26 non-null     float64
 9   avg_study_duration       29 non-null     float64
 10  avg_completion_rating    26 non-null     float64
 11  avg_exam_score           29 non-null     float64
 12  exam_pass_rate           29 non-null     float64
 13  days_since_last_active   31 non-null     int64  
dtypes: float64(7), int64(4), obj

In [514]:
df_final.head()

,user_id,display_name,name,email,phone,total_tracking_events,total_completed_modules,total_submissions,avg_submission_rating,avg_study_duration,avg_completion_rating,avg_exam_score,exam_pass_rate,days_since_last_active
0,96989,igihcksn,Inggih Wicaksono,igihcksn@gmail.com,6.281271e+12,3069,259,58.0,2.258621,32.000000,4.120556,70.817175,0.587258,67
1,938276,nurrizkiadip,Nur Rizki Adi Prasetyo,nrizki@dicoding.com,6.287795e+12,2205,171,59.0,2.372881,28.210526,4.432667,84.150838,0.832402,48
2,5021477,rifath_2SXp,rifath,rifathali088@gmail.com,6.289662e+12,187,0,NaN,NaN,2.000000,NaN,83.086957,0.826087,33
3,5044844,ledis_idola_h8Ge,LEDIS IDOLA,221113142@students.mikroskil.ac.id,6.282312e+12,960,0,6.0,3.833333,35.750000,4.625000,91.056075,0.962617,33
4,5051374,fkaslana,Fircan Ferdinand,kaslanafircan@gmail.com,6.288222e+12,157,0,NaN,NaN,63.500000,NaN,69.371429,0.657143,33


### Ubah Tipe Data

In [515]:
int_cols = [
    "total_tracking_events",
    "total_completed_modules",
    "total_submissions",
    "days_since_last_active"
]

for col in int_cols:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(0).astype(int)

In [516]:
pd.options.display.float_format = None # Reset float display to default

# 1. phone to string with '62' prefix and correct format
def format_phone_with_prefix(phone_num):
    if pd.isna(phone_num):
        return ""
    # Convert float to integer string. Example: 6.281271e+12 -> "6281271000000"
    phone_str = str(int(phone_num)).strip()
    if not phone_str:
        return ""
    # Remove leading '0' if present and it's part of a longer number (e.g., if input was '0812...')
    if phone_str.startswith('0') and len(phone_str) > 1:
        phone_str = phone_str[1:]
    # Add '62' prefix if not already present
    if not phone_str.startswith('62'):
        return '62' + phone_str
    return phone_str

df_final["phone"] = df_final["phone"].apply(format_phone_with_prefix)



In [517]:
# 2. Float columns to 2 decimal places
float_cols_to_round = [
    "avg_submission_rating",
    "avg_study_duration",
    "avg_completion_rating",
    "avg_exam_score",
    "exam_pass_rate"
]

for col in float_cols_to_round:
    if col in df_final.columns:
        df_final[col] = df_final[col].fillna(0).round(2)

In [518]:
df_final.head(32)

,user_id,display_name,name,email,phone,total_tracking_events,total_completed_modules,total_submissions,avg_submission_rating,avg_study_duration,avg_completion_rating,avg_exam_score,exam_pass_rate,days_since_last_active
0,96989,igihcksn,Inggih Wicaksono,igihcksn@gmail.com,6281270768151,3069,259,58,2.26,32.00,4.12,70.82,0.59,67
1,938276,nurrizkiadip,Nur Rizki Adi Prasetyo,nrizki@dicoding.com,6287794555845,2205,171,59,2.37,28.21,4.43,84.15,0.83,48
2,5021477,rifath_2SXp,rifath,rifathali088@gmail.com,6289662167888,187,0,0,0.00,2.00,0.00,83.09,0.83,33
3,5044844,ledis_idola_h8Ge,LEDIS IDOLA,221113142@students.mikroskil.ac.id,6282312102689,960,0,6,3.83,35.75,4.62,91.06,0.96,33
4,5051374,fkaslana,Fircan Ferdinand,kaslanafircan@gmail.com,6288221935189,157,0,0,0.00,63.50,0.00,69.37,0.66,33
5,5181638,anggit_andreansyah,Anggit andreansyah,anggitandreansyah119@gmail.com,6282335590496,13,0,0,0.00,0.00,0.00,0.00,0.00,33
6,5410562,jeni_amanda_ABFC,Jeni Amanda,jeniamandaa@gmail.com,,38,0,0,0.00,0.00,0.00,91.33,1.00,33
7,5410865,ramadhan_oktarizaldi_02Qj,ramadhan oktarizaldi,roktarizaldi@gmail.com,6288202155910,12,0,0,0.00,0.00,0.00,0.00,0.00,33
8,3390,istiabudi73,ISTIA BUDI,istiabudi@gmail.com,628152001234,5526,296,133,1.64,49.96,3.97,79.80,0.72,72
9,5774,budhisant,Budi Santosa,budhisant@gmail.com,6281330533131,2628,258,5,1.40,70.67,3.00,67.02,0.52,45


## **Simpan Dataset**

In [519]:
df_final.to_csv("final_dataset.csv", index=False)
print("final_dataset.csv Berhasil Disimpan!")


final_dataset.csv Berhasil Disimpan!


# **2. Mencari Aktivitas Semua User (Untuk CLustering)**

## MEngambil Kolom yang Dibutuhkan

In [520]:
df_users_s = df_users[["user_id", "display_name", "name", "email", "phone"]]

df_journey_s = df_journey[
    ["journey_id", "name", "difficulty", "hours_to_study", "type", "graduation"]
].rename(columns={"name": "journey_name"})

df_submission_s = df_submission[
    ["id", "user_id", "journey_id", "quiz_id", "status", "rating", "submission_duration"]
].rename(columns={
    "id": "submission_id",
    "status": "submission_status"
})

df_complete_s = df_complete[
    ["user_id", "journey_id", "enrolling_times", "study_duration", "avg_submission_rating"]
]


## Merge Data

In [521]:
df_exam_full = df_regist.merge(
    df_exam,
    how="left",
    left_on="id",
    right_on="regist_id"
)

df_exam_full_s = df_exam_full[
    ["user_id", "tutorial_id", "id_x", "score", "is_passed",
     "total_questions", "exam_finished_at", "status"]
].rename(columns={
    "id_x": "exam_registration_id",
    "status": "exam_status"
})


In [522]:
df_record_full = df_tracking.copy()

df_record_full["first_opened_at"] = pd.to_datetime(
    df_record_full["first_opened_at"], errors="coerce"
)
df_record_full["completed_at"] = pd.to_datetime(
    df_record_full["completed_at"], errors="coerce"
)

df_record_full["time_to_complete_hours"] = (
    (df_record_full["completed_at"] - df_record_full["first_opened_at"])
    .dt.total_seconds() / 3600
)


In [523]:
df_record_full = df_record_full.merge(
    df_users_s,
    on="user_id",
    how="left"
)

df_record_full = df_record_full.merge(
    df_journey_s,
    on="journey_id",
    how="left"
)

In [524]:
df_record_full = df_record_full.merge(
    df_submission_s,
    on=["user_id", "journey_id"],
    how="left"
)

df_record_full = df_record_full.merge(
    df_complete_s,
    on=["user_id", "journey_id"],
    how="left"
)

In [525]:
df_record_full = df_record_full.merge(
    df_exam_full_s,
    on=["user_id", "tutorial_id"],
    how="left"
)

In [526]:
print("SHAPE df_record_full:", df_record_full.shape)
df_record_full.head()

SHAPE df_record_full: (318011, 33)


,id,journey_id,tutorial_id,user_id,status,last_viewed,first_opened_at,completed_at,last_activity_date,time_to_complete_hours,...,submission_duration,enrolling_times,study_duration,avg_submission_rating,exam_registration_id,score,is_passed,total_questions,exam_finished_at,exam_status
0,266063,26,599,96989,1,2017-05-26 21:34:00,NaT,NaT,2017-05-26 21:34:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,362382,26,602,96989,0,2017-05-26 21:35:00,NaT,NaT,2017-05-26 21:35:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2313738,32,1152,96989,1,NaT,2018-08-29 11:19:00,NaT,2018-08-29 11:19:00,NaN,...,0,3.0,44.0,4.5,NaN,NaN,NaN,NaN,NaN,NaN
3,2313738,32,1152,96989,1,NaT,2018-08-29 11:19:00,NaT,2018-08-29 11:19:00,NaN,...,0,3.0,44.0,4.5,NaN,NaN,NaN,NaN,NaN,NaN
4,2314290,32,719,96989,1,NaT,2018-08-29 11:32:00,NaT,2018-08-29 11:32:00,NaN,...,0,3.0,44.0,4.5,NaN,NaN,NaN,NaN,NaN,NaN


In [529]:
df_record_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318011 entries, 0 to 318010
Data columns (total 33 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      318011 non-null  int64         
 1   journey_id              318011 non-null  int64         
 2   tutorial_id             318011 non-null  int64         
 3   user_id                 318011 non-null  int64         
 4   status                  318011 non-null  int64         
 5   last_viewed             27386 non-null   datetime64[ns]
 6   first_opened_at         242262 non-null  datetime64[ns]
 7   completed_at            21970 non-null   datetime64[ns]
 8   last_activity_date      252188 non-null  datetime64[ns]
 9   time_to_complete_hours  18458 non-null   float64       
 10  display_name            318011 non-null  object        
 11  name                    318011 non-null  object        
 12  email                   318011

In [528]:
print("Tracking awal:", df_tracking.shape)
print("Record full:", df_record_full.shape)


Tracking awal: (101736, 9)
Record full: (318011, 33)


In [527]:
df_record_full.to_csv("df_record_full.csv", index=False)
print("df_record_full.csv berhasil disimpan!")


df_record_full.csv berhasil disimpan!
